# 1. Loader

In [16]:
from scipy.io import loadmat
import numpy as np
import mne
mne.set_log_level('WARNING')

def loader_from_matdata(filename):
    matdata = loadmat(filename)
    
    data, info, events = convert_matdata_to_data_info_events(matdata)
    tmin = 0.5
    event_id = {
        'left': -1,
        'right': 1
    }
    epochs = mne.EpochsArray(data, info, events, tmin, event_id, verbose=None)
    epochs.filter(l_freq=8, h_freq=32)
    labels = epochs.events[:,-1]
    return epochs, labels


def convert_matdata_to_data_info_events(matdata):
    sfreq = matdata['nfo']['fs'][0][0][0][0]
    EEGData = matdata['cnt'].T
    n_channels, n_samples = EEGData.shape
    channels_names = [s[0] for s in matdata['nfo']['clab'][0][0][0]]
    info = mne.create_info(
        ch_names = channels_names,
        ch_types = ['eeg']*n_channels,
        sfreq    = sfreq )
    trials = {}
    win = np.arange(int(0.5*sfreq), int(2.5*sfreq))
    cl_lab = [s[0] for s in matdata['nfo']['classes'][0][0][0]]
    events_codes = matdata['mrk'][0][0][1]
    events_onset = matdata['mrk'][0][0][0]
    cl1    = cl_lab[0]
    cl2    = cl_lab[1]
    nsamples = len(win)

    for cl, code in zip(cl_lab, np.unique(events_codes)):
        cl_onsets = events_onset[events_codes == code]
        trials[cl] = np.zeros((n_channels, nsamples, len(cl_onsets)))
        for i, onset in enumerate(cl_onsets):
            trials[cl][:,:,i] = EEGData[:, win+onset]
    left_hand  = np.rollaxis(trials[cl1], 2, 0)
    right_hand = np.rollaxis(trials[cl2], 2, 0)
    data = np.concatenate([left_hand, right_hand])
    Y = np.concatenate([-np.ones(left_hand.shape[0]),
                        np.ones(right_hand.shape[0])])
    len_events = Y.shape[0]
    ev = [i*sfreq*3 for i in range(len_events)]
    events = np.column_stack((np.array(ev,  dtype = int),
                          np.zeros(len_events,  dtype = int),
                          np.array(Y,  dtype = int)))
    return data, info, events

def load_all_data():
    range_numbers = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
    epochs = []
    labels = []
    for range_number in range_numbers:
        filename = '../data/raw_mat/BCICIV_calib_ds1'+range_number+'.mat'
        epochs_, labels_ = loader_from_matdata(filename)
        epochs.append(epochs_)
        labels.append(labels_)
    return mne.concatenate_epochs(epochs), np.concatenate(labels)
epochs, labels = load_all_data()
epochs

<EpochsArray | 1400 events (all good), 0.5 – 2.49 s (baseline off), ~126.1 MB, data loaded,
 'left': 700
 'right': 700>

# 2. Train

In [12]:
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
# Recherche des meilleurs hyperparamètres
from sklearn.model_selection import GridSearchCV
param_grid = {
    'CSP__n_components': [2, 4, 6, 8],
    'LDA__solver': ['svd', 'lsqr']
}

clf = Pipeline([
    ('CSP', CSP()),
    ('LDA', LinearDiscriminantAnalysis())
])

X_train, X_test, y_train, y_test = train_test_split(
    epochs.get_data(),
    labels,
    test_size=0.2,
    random_state=42
)
grid = GridSearchCV(clf, param_grid, cv=5)
grid.fit(X_train, y_train)
print(f"Meilleurs paramètres : {grid.best_params_}")
best_model = grid.best_estimator_
print(f"Score de validation croisée : {grid.best_score_}")

Meilleurs paramètres : {'CSP__n_components': 8, 'LDA__solver': 'svd'}
Score de validation croisée : 0.7321428571428571


In [13]:
from sklearn.metrics import accuracy_score

predictions = best_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, predictions):.3f}")

Accuracy: 0.711
